In [1]:
import gmaps
import pandas as pd
import numpy as np
import requests
import json
from sqlalchemy import create_engine

In [2]:
Path = "winemag-data-130k-v3.csv"
wine_csv_df = pd.read_csv(Path, encoding="UTF-8")
wine_csv_df

,country,province,region_1,region_2,title,variety,winery
0,Italy,Sicily & Sardinia,Etna,NaN,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,Douro,NaN,NaN,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,Oregon,Willamette Valley,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,Michigan,Lake Michigan Shore,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,Spain,Northern Spain,Navarra,NaN,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
6,Italy,Sicily & Sardinia,Vittoria,NaN,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
7,France,Alsace,Alsace,NaN,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
8,Germany,Rheinhessen,NaN,NaN,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel
9,France,Alsace,Alsace,NaN,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam


In [3]:
json_file = "winemag-data-130k-v3.json"
wine_json_df = pd.read_json(json_file)
wine_json_df.head()

,title,description,designation,points,price,taster_name,taster_twitter_handle
0,Nicosia 2013 Vulkà Bianco (Etna),"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Kerin O’Keefe,@kerinokeefe
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Roger Voss,@vossroger
10,Kirkland Signature 2011 Mountain Cuvée Caberne...,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,Virginie Boone,@vboone
100,Ventosa 2015 Pinot Gris (Finger Lakes),"Fresh apple, lemon and pear flavors are accent...",None,88,18.0,Anna Lee C. Iijima,None
1000,Arcane Cellars 2006 Cabernet Sauvignon (Rogue ...,Arcane's Cab is stylistically apart from eithe...,None,88,24.0,Paul Gregutt,@paulgwine


In [4]:
complete_wine_df = pd.merge(wine_json_df, wine_csv_df, on = "title")

complete_wine_df.head()

,title,description,designation,points,price,taster_name,taster_twitter_handle,country,province,region_1,region_2,variety,winery
0,Nicosia 2013 Vulkà Bianco (Etna),"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Kerin O’Keefe,@kerinokeefe,Italy,Sicily & Sardinia,Etna,NaN,White Blend,Nicosia
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Roger Voss,@vossroger,Portugal,Douro,NaN,NaN,Portuguese Red,Quinta dos Avidagos
2,Kirkland Signature 2011 Mountain Cuvée Caberne...,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,Virginie Boone,@vboone,US,California,Napa Valley,Napa,Cabernet Sauvignon,Kirkland Signature
3,Kirkland Signature 2011 Mountain Cuvée Caberne...,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,Virginie Boone,@vboone,US,California,Napa Valley,Napa,Cabernet Sauvignon,Kirkland Signature
4,Ventosa 2015 Pinot Gris (Finger Lakes),"Fresh apple, lemon and pear flavors are accent...",None,88,18.0,Anna Lee C. Iijima,None,US,New York,Finger Lakes,Finger Lakes,Pinot Gris,Ventosa


In [5]:
complete_wine_df = complete_wine_df.drop(columns="region_2")

complete_wine_df.head()

,title,description,designation,points,price,taster_name,taster_twitter_handle,country,province,region_1,variety,winery
0,Nicosia 2013 Vulkà Bianco (Etna),"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Kerin O’Keefe,@kerinokeefe,Italy,Sicily & Sardinia,Etna,White Blend,Nicosia
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Roger Voss,@vossroger,Portugal,Douro,NaN,Portuguese Red,Quinta dos Avidagos
2,Kirkland Signature 2011 Mountain Cuvée Caberne...,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,Virginie Boone,@vboone,US,California,Napa Valley,Cabernet Sauvignon,Kirkland Signature
3,Kirkland Signature 2011 Mountain Cuvée Caberne...,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,Virginie Boone,@vboone,US,California,Napa Valley,Cabernet Sauvignon,Kirkland Signature
4,Ventosa 2015 Pinot Gris (Finger Lakes),"Fresh apple, lemon and pear flavors are accent...",None,88,18.0,Anna Lee C. Iijima,None,US,New York,Finger Lakes,Pinot Gris,Ventosa


In [6]:
complete_wine_df = complete_wine_df.dropna()

In [7]:
complete_wine_df.drop_duplicates(subset = "title", inplace = True) 

complete_wine_df.head()

,title,description,designation,points,price,taster_name,taster_twitter_handle,country,province,region_1,variety,winery
2,Kirkland Signature 2011 Mountain Cuvée Caberne...,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,Virginie Boone,@vboone,US,California,Napa Valley,Cabernet Sauvignon,Kirkland Signature
9,Henriet-Bazin NV Blanc de Noirs Grand Cru Brut...,From Verzenay on the Montagne de Reims this wi...,Blanc de Noirs Grand Cru Brut,91,68.0,Roger Voss,@vossroger,France,Champagne,Champagne,Pinot Noir,Henriet-Bazin
16,M. Chapoutier 1999 Le Méal Ermitage (Hermitage),Chapoutier's selections of the best parcels of...,Le Méal Ermitage,98,150.0,Roger Voss,@vossroger,France,Rhône Valley,Hermitage,Rhône-style Red Blend,M. Chapoutier
21,E. Guigal 1998 La Mouline (Côte Rôtie),Even though the 42 months this wine has spent ...,La Mouline,96,175.0,Roger Voss,@vossroger,France,Rhône Valley,Côte Rôtie,Rhône-style Red Blend,E. Guigal
23,E. Guigal 1998 La Turque (Côte Rôtie),"A firm, dry, foursquare wine that has power an...",La Turque,95,175.0,Roger Voss,@vossroger,France,Rhône Valley,Côte Rôtie,Rhône-style Red Blend,E. Guigal


In [8]:
complete_wine_df.insert(0, 'id', range(1, 1 + len(complete_wine_df)))

complete_wine_df.head()

,id,title,description,designation,points,price,taster_name,taster_twitter_handle,country,province,region_1,variety,winery
2,1,Kirkland Signature 2011 Mountain Cuvée Caberne...,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,Virginie Boone,@vboone,US,California,Napa Valley,Cabernet Sauvignon,Kirkland Signature
9,2,Henriet-Bazin NV Blanc de Noirs Grand Cru Brut...,From Verzenay on the Montagne de Reims this wi...,Blanc de Noirs Grand Cru Brut,91,68.0,Roger Voss,@vossroger,France,Champagne,Champagne,Pinot Noir,Henriet-Bazin
16,3,M. Chapoutier 1999 Le Méal Ermitage (Hermitage),Chapoutier's selections of the best parcels of...,Le Méal Ermitage,98,150.0,Roger Voss,@vossroger,France,Rhône Valley,Hermitage,Rhône-style Red Blend,M. Chapoutier
21,4,E. Guigal 1998 La Mouline (Côte Rôtie),Even though the 42 months this wine has spent ...,La Mouline,96,175.0,Roger Voss,@vossroger,France,Rhône Valley,Côte Rôtie,Rhône-style Red Blend,E. Guigal
23,5,E. Guigal 1998 La Turque (Côte Rôtie),"A firm, dry, foursquare wine that has power an...",La Turque,95,175.0,Roger Voss,@vossroger,France,Rhône Valley,Côte Rôtie,Rhône-style Red Blend,E. Guigal


In [9]:
rds_connection_string = "postgres:primero1490@localhost:5432/wine_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

engine.table_names()

['complete_wine']

In [10]:
complete_wine_df.to_sql(name='complete_wine', \
                            con=engine, if_exists="append", index=False)